In [3]:
import sys
import os

# Add the project root directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [4]:
import numpy as np
from pandas import DataFrame


def transform(df: DataFrame) -> DataFrame:
    df['embedding'] = df.apply(lambda row: np.concatenate((row['main_pic_embeddings_resnet_v1'][0], row['name_bert_64'], row['categories'], row['characteristic_attributes_mapping'])), axis=1)
    return df[['variantid', 'embedding']]

In [5]:
from src.data_loading import load_data_batch
import pyarrow as pa

df = load_data_batch("../data/train/merged_data_prikol.parquet")

In [6]:
from src.parquet_zavod import parquet_zavod


# parquet_zavod(
#     '../data/train/merged_data_prikol.parquet',
#     '../data/train/merged_data_embedding.parquet',
#     pa.Table.from_pandas(transform(df)).schema,
#     transform,
# )

Processing: 100%|██████████| 2252569/2252569 [01:22<00:00, 27440.60rows/s]


In [10]:
from src.data_loading import load_data_full


df_train = load_data_full('../data/train/train.parquet')
df = load_data_full('../data/train/merged_data_embedding.parquet')

In [11]:
import pandas as pd

def merger(df_train, df):
    # Step 1: Merge df_train with df for variantid1
    df_merged1 = pd.merge(
        df_train, df, left_on="variantid1", right_on="variantid", how="left"
    )

    # Step 2: Rename the embedding column
    df_merged1 = df_merged1.rename(columns={"embedding": "embedding1"})

    # Step 3: Merge again with df for variantid2
    df_final = pd.merge(
        df_merged1, df, left_on="variantid2", right_on="variantid", how="left"
    )

    # Step 4: Rename the second embedding column
    df_final = df_final.rename(columns={"embedding": "embedding2"})

    # Step 5: Select only the columns we need
    return df_final[["variantid1", "variantid2", "embedding1", "embedding2", "target"]]

In [12]:
dff = merger(df_train, df)
dff

,variantid1,variantid2,embedding1,embedding2,target
0,1447875869,1447872068,"[0.5318107604980469, 0.35363996028900146, -0.7...","[0.5318107604980469, 0.35363996028900146, -0.7...",1
1,1176231201,284733670,"[0.4308440089225769, 0.7620932459831238, 0.793...","[0.5668608546257019, 0.9573432803153992, 1.017...",1
2,658617865,549848659,"[-0.36238163709640503, 0.4316844344139099, -0....","[-0.25123998522758484, 0.3757574260234833, -0....",0
3,719320625,719370486,"[0.7327960729598999, -0.7488707900047302, 0.55...","[0.7327960729598999, -0.7488707900047302, 0.55...",1
4,1067645658,949954740,"[-1.3140270709991455, -0.8071212768554688, 0.7...","[-0.49589139223098755, -0.5760805606842041, 0....",0
...,...,...,...,...,...
1168511,970087885,944020046,"[-0.5370045900344849, 0.14626435935497284, 0.4...","[-0.5370045900344849, 0.14626435935497284, 0.4...",0
1168512,834554806,118393780,"[0.17166948318481445, 0.2693502604961395, 0.05...","[0.17166948318481445, 0.2693502604961395, 0.05...",1
1168513,1473905394,1473908406,"[-0.7199546098709106, -0.22784380614757538, -0...","[-0.7199546098709106, -0.22784380614757538, -0...",1
1168514,104892989,1549506903,"[-0.560542643070221, -0.8331247568130493, 0.28...","[-0.561845064163208, -0.8333196640014648, 0.28...",1


In [13]:
dff.to_parquet('../data/train/train_data.parquet')